# Retrieve CO-OPS data from an arbitrary time interval and region

In [ ]:
import searvey

In [ ]:
import geopandas
from matplotlib import pyplot

countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# retrieve a [list of CO-OPS stations](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations)

In [ ]:
from searvey.coops import coops_stations

stations = coops_stations()
stations

In [ ]:
figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

stations.plot(ax=axis)
countries.plot(color='lightgrey', ax=axis, zorder=-1)

axis.set_title(f'all CO-OPS stations')

# retrieve a [list of CO-OPS stations within an arbitrary polygon](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations_within_region)

In [ ]:
import shapely
from searvey.coops import coops_stations_within_region

east_coast = shapely.geometry.box(-85, 25, -65, 45)
east_coast_stations = coops_stations_within_region(region=east_coast)
east_coast_stations

In [ ]:
figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

east_coast_stations.plot(ax=axis)

xlim = axis.get_xlim()
ylim = axis.get_ylim()
countries.plot(color='lightgrey', ax=axis, zorder=-1)
axis.set_xlim(xlim)
axis.set_ylim(ylim)

axis.set_title(f'CO-OPS stations on the U.S. East Coast')

# retrieve a [CO-OPS data product from all stations within an arbitrary polygon](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_product_within_region)

In [ ]:
from datetime import datetime
from searvey.coops import coops_product_within_region

water_levels = coops_product_within_region(
    'water_level',
    region=east_coast,
    start_date=datetime(2022, 4, 2, 12),
    end_date=datetime(2022, 4, 2, 12, 30),
)
water_levels

In [ ]:
import pandas

figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

water_levels.max('t').plot.scatter(x='x', y='y', hue='v', cmap='viridis', ax=axis)

xlim = axis.get_xlim()
ylim = axis.get_ylim()
countries.plot(color='lightgrey', ax=axis, zorder=-1)
axis.set_xlim(xlim)
axis.set_ylim(ylim)

start_date = pandas.to_datetime(water_levels['t'].min().values)
end_date = pandas.to_datetime(water_levels['t'].max().values)
axis.set_title(f'maximum water levels observed by CO-OPS stations on the U.S. East Coast between {start_date} and {end_date}')

# retrieve a CO-OPS data product from a [specific station](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.COOPS_Station)

In [ ]:
from searvey.coops import COOPS_Station

station = COOPS_Station(1612480)
station_water_levels = station.product(
    'water_level',
    start_date=datetime(2022, 4, 2, 12),
    end_date=datetime(2022, 4, 2, 12, 30),
)
station_water_levels

In [ ]:
figure, axis = pyplot.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

axis.errorbar(
    station_water_levels['t'],
    station_water_levels['v'].isel(nos_id=0),
    yerr=station_water_levels['s'].isel(nos_id=0).values,
)

start_date = pandas.to_datetime(station_water_levels['t'].min().values)
end_date = pandas.to_datetime(station_water_levels['t'].max().values)
axis.set_title(f'water level observed by CO-OPS station {station.id} ("{station.name}") between {start_date} and {end_date}')

# additional documentation
## https://searvey.readthedocs.io
## https://github.com/oceanmodeling/searvey
